In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [8]:
df2022 = pd.read_csv("annual campaign/2022 Annual Campaign - Active.csv")
df2023 = pd.read_csv("annual campaign/2023 Annual Campaign - Active.csv")

In [9]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Filter for donations
data = df2022.dropna(subset=['City', 'Donation Amount'])  # Drop rows with missing data

# Aggregate donations by city
aggregated_donations = data.groupby('City')['Donation Amount'].sum().reset_index()

# Initialize geolocator
geolocator = Nominatim(user_agent="donation_heatmap")

# Geocode function to get latitude and longitude for each city
def geocode_city(city, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(f"{city}, USA", timeout=10)  # Increase timeout
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except GeocoderTimedOut:
            if attempt < retries - 1:
                time.sleep(2)  # Wait before retrying
                continue
            return None, None

# Add latitude and longitude columns
aggregated_donations[['Latitude', 'Longitude']] = aggregated_donations['City'].apply(
    lambda city: pd.Series(geocode_city(city))
)

# Drop rows where geocoding failed
aggregated_donations = aggregated_donations.dropna(subset=['Latitude', 'Longitude'])

# Create the base map
base_map2022 = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

# Prepare data for the heat map (latitude, longitude, donation amount)
heat_data = aggregated_donations[['Latitude', 'Longitude', 'Donation Amount']].values.tolist()

# Add the heat map to the base map
HeatMap(heat_data, radius=15).add_to(base_map2022)

# Save and display the map
base_map2022.save("donations_heatmap.html")
base_map2022
#this is the heat map from the 2022 donations dataset

In [10]:
# Filter for donations
data2 = df2023.dropna(subset=['City', 'Donation Amount'])  # Drop rows with missing data

# Aggregate donations by city
aggregated_donations2 = data2.groupby('City')['Donation Amount'].sum().reset_index()

# Initialize geolocator
geolocator = Nominatim(user_agent="donation_heatmap")

# Geocode function to get latitude and longitude for each city
def geocode_city(city, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(f"{city}, USA", timeout=10)  # Increase timeout
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except GeocoderTimedOut:
            if attempt < retries - 1:
                time.sleep(2)  # Wait before retrying
                continue
            return None, None

# Add latitude and longitude columns to the aggregated donations DataFrame
aggregated_donations2[['Latitude', 'Longitude']] = aggregated_donations2['City'].apply(
    lambda city: pd.Series(geocode_city(city))
)

# Drop rows where geocoding failed
aggregated_donations2 = aggregated_donations2.dropna(subset=['Latitude', 'Longitude'])

# Create the base map
base_map2023 = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

# Prepare data for the heat map (latitude, longitude, donation amount)
heat_data2 = aggregated_donations2[['Latitude', 'Longitude', 'Donation Amount']].values.tolist()

# Add the heat map to the base map
HeatMap(heat_data2, radius=15).add_to(base_map2023)

# Save and display the map
base_map2023.save("donations_heatmap_2023.html")
base_map2023